In [ ]:
#!pip install sentence-transformers

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.8 MB/s eta 0:00:00


In [ ]:
# Install packages
!pip install nltk

In [ ]:
import numpy as np
from scipy.stats import pearsonr
from google.colab import drive
import re
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Viene inserito la tokenizzazione e poi lo stemming, provare anche con lemmatizzazione

In [ ]:
import nltk
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

nltk.download('stopwords')  # Download nltk stopwords
#nltk.download('punkt')      # Download nltk punkt tokenizer
#nltk.download('wordnet')

text = []
stopwords_list = set(stopwords.words("english"))

#lemmatizer = WordNetLemmatizer()

with open('/content/drive/MyDrive/deep_learning/progetto_deep/Semantic Textual Similarity/Clinical Semantic Textual Similarity/clinicalSTS2019/train/clinicalSTS2019.train.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip()

        # Splitting by tab
        line_parts = re.split('\t', line)

        filtered_sentences = []
        for sentence in line_parts:
            # Lowercasing
            lower_sentence = sentence.lower()

            # Removing punctuation except periods
            processed_sentence = re.sub(r'[^\w\s.]', '', lower_sentence)

            # Replace dots between numbers with commas
            processed_sentence = re.sub(r'(?<=\d)\.(?=\d)', ',', processed_sentence)

            # Remove remaining dots
            processed_sentence = processed_sentence.replace('.', '')

            # Replace dots between numbers with dots
            processed_sentence = re.sub(r'(?<=\d),(?=\d)', '.', processed_sentence)

            tokenized_sentence = tokenizer.tokenize(processed_sentence)

            # Rimozione delle stopwords
            filtered_tokens = [token for token in tokenized_sentence if token.lower() not in stopwords_list]

            filtered_sentences.append(filtered_tokens)

        text.append(filtered_sentences)

Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
#text[0]

In [ ]:
def make_split(embedded_representation, labels):
    X_train, X_test, y_train, y_test = train_test_split(embedded_representation, labels, random_state=104, test_size=0.25, shuffle=True)

    y_train = np.array(y_train, dtype=object)
    y_test = np.array(y_test, dtype=object)

    X_train = np.array(X_train, dtype=object)
    X_test = np.array(X_test, dtype=object)

    return X_train, X_test, y_train, y_test

In [ ]:
def cosine_similarity_unidimensional(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

In [ ]:
def normalize_train(y):
  y = [float(val) for val in y]  # Converti i valori in numeri decimali
  min_val = np.min(y)
  max_val = np.max(y)
  normalized_labels_train = (y - min_val) / (max_val - min_val) * 2 - 1

  return normalized_labels_train



In [ ]:
# Definisci una lista di modelli da provare
models = [
    'bert-base-nli-mean-tokens',
    'bert-base-nli-max-tokens',
    'bert-base-nli-cls-token'
]

all_results={}

for model_name in models:
    # Carica il modello
    model = SentenceTransformer(model_name)
    text_embedded=[]
    label=[]
    train_cosine_similarity = []

    for phrases in range(len(text)):
      sentence_vectors = model.encode(text[phrases][0:2])
      text_embedded.append(sentence_vectors)
      label.append((text[phrases][2]))

    #Print the embeddings
    '''
    for sentence, embedding in zip(text, text_embedded):
      print("Sentence:", sentence)
      print("Embedding:", embedding)
      print()
    '''

    X_train, X_test, y_train, y_test = make_split(text_embedded, label)

    for sentence in X_train:
      sentence1 = sentence[0]
      sentence2 = sentence[1]
      similarity = cosine_similarity_unidimensional(sentence1, sentence2)
      train_cosine_similarity.append(similarity)

    normalized_labels_train= normalize_train(y_train)

    # Calcolo della correlazione di Pearson
    correlation, p_value = pearsonr(train_cosine_similarity, normalized_labels_train)

    # Salva i risultati nel dizionario
    all_results[model_name] = {
        #'text_embedded': text_embedded,
        #'label': label,
        #'train_similarity': train_cosine_similarity,
        'correlation': correlation,
        'p_value': p_value
    }